In [ ]:
import numpy as np
import pandas as pd
import string


In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/dataset/data.csv')
tempo = np.array(dataset[['text','label_num']])

X = np.array(tempo[:,0])
y = np.array(tempo[:,-1]).astype('float32')


In [ ]:
#lower case
#removing Punctuation
transTable = str.maketrans(dict.fromkeys(string.punctuation))
X = np.array([sen.lower().translate(transTable) for sen in X])

import re
def remove_hyperlink(word):
    return re.sub(r"http\S+", "", word)

X = np.array([remove_hyperlink(sen) for sen in X])


In [ ]:
max_feature = 50000
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=max_feature)
tokenizer.fit_on_texts(X)
X_features =  np.array(tokenizer.texts_to_sequences(X))

max_len = 0;
for arr in X_features:
    max_len = max(max_len,len(arr))

from keras.preprocessing.sequence import pad_sequences
X_features = pad_sequences(X_features,maxlen=max_len,padding='post')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


In [ ]:
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test = tts(X_features, y, test_size=0.33, random_state=42,shuffle=True)
X_test, X_valid, y_test, y_valid = tts(X_test, y_test, test_size=0.5, random_state=42,shuffle=True)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,LSTM, Embedding, Dropout, Activation, Bidirectional
#size of the output vector from each layer
embedding_vector_length = 32
#Creating a sequential model
model = tf.keras.Sequential()
#Creating an embedding layer to vectorize
model.add(Embedding(max_feature, embedding_vector_length, input_length=max_len))
#Addding Bi-directional LSTM
model.add(Bidirectional(tf.keras.layers.LSTM(64)))
#Relu allows converging quickly and allows backpropagation
model.add(Dense(16, activation='relu'))
#Deep Learninng models can be overfit easily, to avoid this, we add randomization using drop out
model.add(Dropout(0.1))
#Adding sigmoid activation function to normalize the output
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3955, 32)          1600000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               49664     
_________________________________________________________________
dense_2 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 1,651,745
Trainable params: 1,651,745
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history = model.fit(X_train,y_train,batch_size = 200,epochs=20,validation_data=(X_valid,y_valid),verbose=True)

Epoch 1/20
18/18 [==============================] - 318s 17s/step - loss: 0.6773 - accuracy: 0.6311 - val_loss: 0.5677 - val_accuracy: 0.7272
Epoch 2/20
18/18 [==============================] - 296s 17s/step - loss: 0.5665 - accuracy: 0.6997 - val_loss: 0.3450 - val_accuracy: 0.7272
Epoch 3/20
18/18 [==============================] - 293s 16s/step - loss: 0.4169 - accuracy: 0.7555 - val_loss: 0.2612 - val_accuracy: 0.9274
Epoch 4/20
18/18 [==============================] - 282s 16s/step - loss: 0.2069 - accuracy: 0.9670 - val_loss: 0.1108 - val_accuracy: 0.9567
Epoch 5/20
18/18 [==============================] - 277s 15s/step - loss: 0.0542 - accuracy: 0.9870 - val_loss: 0.0748 - val_accuracy: 0.9766
Epoch 6/20
18/18 [==============================] - 288s 16s/step - loss: 0.0208 - accuracy: 0.9951 - val_loss: 0.1140 - val_accuracy: 0.9684
Epoch 7/20
18/18 [==============================] - 296s 17s/step - loss: 0.0264 - accuracy: 0.9942 - val_loss: 0.0752 - val_accuracy: 0.9742
Epoch 

In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets
INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [ ]:
model.save("saved_model/mask_detector.model", save_format="h5")

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_pred = [1 if o>0.5 else 0 for o in y_pred]

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix,f1_score, precision_score,recall_score
cf_matrix =confusion_matrix(y_test,y_pred)
tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
print("Precision: {:.2f}%".format(100 * precision_score(y_test,y_pred)))
print("Recall: {:.2f}%".format(100 * recall_score(y_test,y_pred)))
print("F1 Score: {:.2f}%".format(100 * f1_score(y_test,y_pred)))

Precision: 98.21%
Recall: 96.05%
F1 Score: 97.12%


[[621   4]
 [  9 219]]
